<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/LLM_mit_LangChain/blob/main/LLM_Agent_Simulate_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you start**  
- Contents is always your best friend:P

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('openAI_Key')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai
    !pip install langgraph

# **1. GPT-related method**  
- Using GPT4.0-mini mit Langchain(A library uniforms the syntax of all LLMs)
- Generating Persona(Characters) by LLM
  - Few-shot learning
  - Work perfectly with this prompt

In [ ]:
import time
import random

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import PromptTemplate

def temp_sleep(seconds=0.1):
  """
  Simulate the realistic delay
  """
  time.sleep(seconds)

def GPT4_request(prompt:str):
  """
  Given a prompt, make a request to OpenAI server and returns the response.
  ARGS:
    prompt: a str prompt
  RETURNS:
    a str of GPT-4's response.
  """
  MODEL = 'gpt-4o-mini'
  messages = [
    SystemMessage(content="user"),
    HumanMessage(content= prompt),
    ]

  llm = ChatOpenAI(
    model=MODEL,
    temperature= 0.0,
    n= 1,
    max_tokens= 256)

  return llm.invoke(messages).content

def GPT4_generate_persona(name:str):
  MODEL = 'gpt-4o-mini'
  prompt = f"""
  Please creating information for {name} following the persona template. Don't add extra words.

  ----------------------------------------
  Template:
      name (str): The name of the persona.
      age (int): Age of the persona.
      gender (str): Gender of the persona.
      status (str): Occupation or current status.
      hobbies (list): List of hobbies the persona enjoys.
      wealth (int): Wealth level (0 to 1000).
      favorite_food (str): Persona's favorite food.
      nemesis (str or None): Persona's nemesis, if any.
      quirky_trait (str or None): A quirky personality trait.
  ----------------------------------------
  Example:
        name="Alex",
        age=29,
        gender="Male",
        status="Freelance Illustrator",
        hobbies=["painting", "cycling", "reading sci-fi"],
        wealth=550,
        favorite_food="Sushi",
        nemesis="Karen from accounting",
        quirky_trait="always wearing mismatched socks"

        name="Jill",
        age=27,
        gender="Female",
        status="Software Developer",
        hobbies=["hiking", "playing video games", "cooking"],
        wealth=600,
        favorite_food="Tacos",
        nemesis="The bug in her code",
        quirky_trait="collects vintage keychains"

  """


  messages = [
    SystemMessage(content="You're a system creating a simulated person"),
    HumanMessage(content= prompt),
    ]

  llm = ChatOpenAI(
    model=MODEL,
    temperature= 0.3,
    n= 1,
    max_tokens= 256)

  return llm.invoke(messages).content


#Debug ---- remove content in return line to see metadata
# print(GPT4_request("Hiii, what is your name?"))

# **2. Persona**
- Create an object Persona documenting the personal info
- Generating Persona(Characters) by LLM
  - Few-shot learning
  - Work perfectly with this prompt

In [ ]:
class Persona:
    def __init__(self, name, age, gender, status, hobbies, wealth, favorite_food="Pizza", nemesis=None, quirky_trait=None):
        """
        Initializes a Persona instance with a mix of essential and fun attributes.

        Args:
            name (str): The name of the persona.
            age (int): Age of the persona.
            gender (str): Gender of the persona.
            status (str): Occupation or current status.
            hobbies (list): List of hobbies the persona enjoys.
            wealth (int): Wealth level (0 to 1000).
            favorite_food (str): Persona's favorite food (default is "Pizza").
            nemesis (str or None): Persona's nemesis, if any.
            quirky_trait (str or None): A quirky personality trait.
        """
        self.name = name
        self.age = age
        self.gender = gender
        self.status = status
        self.hobbies = hobbies
        self.wealth = max(0, min(wealth, 1000))  # Clamped between 0 and 1000
        self.favorite_food = favorite_food
        self.nemesis = nemesis
        self.quirky_trait = quirky_trait

    def generate_persona_from_gpt(name:str):
      gpt_response = GPT4_generate_persona(name)
      persona_args = eval(f"dict({gpt_response})")
      return Persona(**persona_args)

    def display_info(self):
        """Displays basic information about the persona."""
        print(f"Name: {self.name}")
        print(f"Age: {self.age}")
        print(f"Gender: {self.gender}")
        print(f"Status: {self.status}")
        print(f"Hobbies: {', '.join(self.hobbies)}")
        print(f"Wealth: {self.wealth}")
        print(f"Favorite Food: {self.favorite_food}")
        print(f"Nemesis: {self.nemesis}")
        print(f"Quirky Trait: {self.quirky_trait}")

    def info_to_dict(self):
      """Returns a dictionary containing the persona's attributes."""
      return {
          "name": self.name,
          "age": self.age,
          "gender": self.gender,
          "status": self.status,
          "hobbies": self.hobbies,
          "wealth": self.wealth,
          "favorite_food": self.favorite_food,
          "nemesis": self.nemesis,
          "quirky_trait": self.quirky_trait,
      }

    def info_to_string_story(self):
      return f'I am {self.name}. My age is {self.age}. My gender is {self.gender}. My status is {self.status}. My hobbies are {self.hobbies}.\
      My wealth is {self.wealth}. My Favorite food is {self.favorite_food}. My nemesis is {self.nemesis}. My Quirky Trait is {self.quirky_trait}'

    def upgrade_wealth(self, amount):
      """
      Increases the persona's wealth by a specified amount, ensuring it stays within the range of 0 to 1000.

      Args:
          amount (int): The amount to add to the persona's wealth.

      Returns:
          None
      """
      if amount < 0:
          print(f"Cannot upgrade wealth by a negative amount: {amount}")
          return
      previous_wealth = self.wealth
      self.wealth = min(self.wealth + amount, 1000)

# **3. Two AI talk to each other**

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from IPython.display import display_markdown


def begin_conversation(persona_name, opposite_persona_name, persona_info, opposite_persona_info):
    MODEL = 'gpt-4o-mini'

    system_message = (
    f"You are {persona_name}. Here is your background:\n"
    f"{persona_info}\n\n"
    f"You are speaking with {opposite_persona_name}, whose background is:\n"
    f"{opposite_persona_info}\n\n"
    "Guidelines:\n"
    "1. Format answers with markdown.\n"
    "2. Answer based on the background information provided.\n"
    f"3. Repeat your name before answering (e.g., \"{persona_name}: ...\").\n"
    "4. Stick to the original topic.\n"
    "5. Avoid repeating yourself.\n")


    # Create system and human message templates
    system_message_template = SystemMessagePromptTemplate.from_template(
        system_message
    )


    human_message_template = HumanMessagePromptTemplate.from_template(
        """
        Conversation: {history}
        Human: {input}
        """
    )

    # Combine into a ChatPromptTemplate
    prompt_template = ChatPromptTemplate.from_messages(
        [system_message_template, human_message_template]
    )

    # Initialize the OpenAI LLM
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1,
        max_tokens= 100
    )

    # Initialize memory with auto-summarization
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=300)

    # Build a conversation chain
    conversation = ConversationChain(
        prompt=prompt_template,
        llm=llm,
        memory=memory,
        verbose=False,
    )

    return conversation


def chat(conversation, prompt, name):
    """
    Simulate a single conversational turn.

    Args:
        conversation: The ConversationChain object.
        prompt (str): The message from the human.
        name (str): The name of the persona responding.

    Returns:
        str: The AI's response.
    """
    # Pass the prompt to the conversation chain
    response = conversation.invoke({"input": prompt})
    response_text = response['response']

    # Display the response in markdown
    display_markdown(f"{response_text}", raw=True)
    return response_text


def persona_to_persona_chat(persona_1_name, persona_2_name, persona_1_info, persona_2_info, persona_2_intro, rounds=5):
    """
    Simulates a conversation between two personas using ConversationChain.

    Args:
        persona_1_name (str): The name of the first persona.
        persona_2_name (str): The name of the second persona.
        persona_2_intro (str): The introductory message from Persona 2.
        rounds (int): Number of conversational exchanges.
    """
    # Initialize two separate conversations for each persona
    persona_1_conversation = begin_conversation(persona_1_name, persona_2_name, persona_1_info, persona_2_info)
    persona_2_conversation = begin_conversation(persona_2_name, persona_1_name, persona_2_info, persona_1_info)

    persona_2_intro = f"{persona_2_name}: {persona_2_intro}."

    # Start the conversation
    print("=== Conversation Start ===")
    persona_1_response = chat(persona_1_conversation, persona_2_intro, persona_1_name)

    # persona_1_response = f"{persona_2_name}: {persona_2_intro}\n {persona_1_response}"
    persona_1_response = f"{persona_2_name}: {persona_2_intro}\n {persona_1_name}: {persona_1_response}."
    for i in range(rounds):
        persona_2_response = chat(persona_2_conversation, persona_1_response, persona_2_name)
        persona_1_response = chat(persona_1_conversation, persona_2_response, persona_1_name)
    print("=== Conversation End ===")


# **4. Run here!**

In [ ]:
persona_lea = Persona.generate_persona_from_gpt("Lea")
persona_alex = Persona(
    name="Alex",
    age=29,
    gender="Male",
    status="Freelance Illustrator",
    hobbies=["painting", "cycling", "reading sci-fi"],
    wealth=550,
    favorite_food="Sushi",
    nemesis="Karen from accounting",
    quirky_trait="always wearing mismatched socks"
)
Persona.display_info(persona_lea)

Name: Lea
Age: 32
Gender: Female
Status: Graphic Designer
Hobbies: photography, traveling, yoga
Wealth: 450
Favorite Food: Pasta
Nemesis: The printer that never works
Quirky Trait: talks to her plants


## **4.1 Case 1: Asking Alex to provide more money than he has**

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 1000 from you?"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
persona_to_persona_chat(persona_1_name, persona_2_name, persona_1_info, persona_2_info, persona_2_intro, rounds=2)
# persona_to_persona_chat(persona_1_name, persona_2_name, persona_2_intro, rounds=5)

=== Conversation Start ===


Alex: I wish I could help, but my wealth is only 550 right now. Maybe we can brainstorm some other ways to get you the funds you need?

Lea: I appreciate the offer, Alex! Maybe we could think about some freelance projects together or even sell some of our artwork online. What do you think?

Alex: That sounds like a great idea, Lea! Collaborating on freelance projects could definitely help us both out financially. Plus, selling our artwork online could reach a wider audience. Do you have any specific ideas in mind for the projects?

Lea: I'm glad you like the idea, Alex! I was thinking we could create a series of themed illustrations that combine your painting style with my graphic design skills. We could also consider making some prints or merchandise featuring our artwork. What themes do you think would resonate with people?

Alex: I love that concept, Lea! Combining our styles could create something really unique. As for themes, how about exploring sci-fi elements since I enjoy reading it? We could also consider nature themes, which could tie in nicely with your photography. What do you think?

=== Conversation End ===


## **4.2 Borrow reasonable money from Alex**

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 20 from you?"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
persona_to_persona_chat(persona_1_name, persona_2_name, persona_1_info, persona_2_info, persona_2_intro, rounds=2)

=== Conversation Start ===


Alex: Sure, Lea! I can lend you 20. Just let me know when you can pay me back.

Lea: Thanks, Alex! I appreciate it. I'll make sure to pay you back soon. Maybe after my next freelance project wraps up!

Alex: No problem, Lea! I'm glad to help. Good luck with your freelance project! If you need any tips or ideas, feel free to ask.

Lea: Thanks, Alex! I really appreciate your support. I might take you up on that offer for tips. Your illustrations are always so inspiring!

Alex: Thanks, Lea! That means a lot to me. I'm always happy to share ideas or techniques. If you ever want to collaborate or brainstorm, just let me know!

=== Conversation End ===


# **98. What to do next**

1. Retrieve new info from a conversation, and then upgrade the Persona.

# **99. Working History**

- 2024.12.16 ---- Rewrite the prompt in SystemMessage(GPT4_generate_persona)